# Elementy programowania funkcyjnego w Pythonie - c.d.
Ostatnio poznaliśmy funkcję `reduce` z modułu `functools`, która aplikuje funkcję dwuargumentową (np. mnożenie) do elementów "listy" kolejno: startowy z pierwszy, wynik z drugim, wynik z trzecim itd. Jej koleżanka: funkcja `accumulate` z modułu `itertools` robi dokładnie to samo, tylko jej wynikiem jest iterator wszystkich pośrednich wartości.

In [ ]:
from functools import *
from itertools import *
l = reduce(lambda x,y: x*y, [2,3,1,7], 10)
print(l)
l = accumulate([2,3,1,7], lambda x,y: x*y, initial=10)
print(*l)

Tak jak w funkcji `reduce` można nie podawać początkowej wartości (wtedy zaczynamy od pierwszego elementu listy) i inaczej niż w funkcji `reduce` możemy nie podawać funkcji, wtedy domyślną funkcją będzie dodawanie.

In [ ]:
l = accumulate([3,20,100,4000])
print(*l)

Wynikiem accumulate jest generator, podobny do tego, zwracanego przez `map` czy `filter`.

## Inne funkcje z 'itertools'

Oprócz `accumulate` i poznanej wcześniej funkcji `islice`, dającej iteratorom funkcjonalność zbliżoną do slice'owania list i napisów, w module `itertools` jest wiele funkcji tworzących i przetwarzających iteratory.
Najlepiej przejrzeć tabelkę na samym początku [dokumentacji tego modułu](https://docs.python.org/3/library/itertools.html). Większość z tych funkcji nie wymaga komentarza, ale kilka omówimy.

Funkcja `starmap` działa tak jak `map`, tylko, że zakłada, że elementami iteratora są ciągi, które dajemy funkcji jako zestawy argumentów, czyli jeśli iterator to `(c0, c1, ...)` to wynikiem jest `(f(*c0), f(*c1), ...)`. W najprostszym (i pewnie najczęstszym) przypadku pozwala to na zaaplikowanie funkcji dwuargumentowej do każdej pary z listy par: 

In [ ]:
lista_par = [(i,i+1) for i in range(10)]
# print(list(map(lambda x,y: x*y, lista_par)))     # to nie zadziała
list(starmap(lambda x,y: x*y, lista_par))        # a to działa

Ciekawszy przypadek polega na aplikowaniu funkcji przyjmującej dowolną liczbę argumentów (np. `mult` poniżej) do kolejnych zestawów danych, być może różnych rozmiarów.

In [ ]:
coraz_dłuższe_krotki = accumulate(range(1,7), lambda k, x: k + (x,), initial=tuple())
#print(*coraz_dłuższe_krotki)     # to oczywiście psuje iterator :)
def mult(*args):
    return reduce(lambda x,y: x*y, args, 1)
list(starmap(mult, coraz_dłuższe_krotki))

Przy próbie odkomentowania linii zawierającej `print(...)` okazuje się, że ciąg dalszy przestaje działać. Dzieje się tak dlatego, że iteratory są "jednorazowe". Aby wykorzystać dany iterator dwukrotnie, należy go rozdwoić za pomocą funkcji `tee`. Przy okazji przykład użycia jeszcze jednej funkcji, `groupby`:

In [ ]:
coraz_dłuższe_krotki = accumulate(range(1,7), lambda k, x: k + (x,), initial=tuple())  # krotki
krotki1, krotki2 = tee(coraz_dłuższe_krotki)  # rozdwojenie
print(*krotki1)                               # wypisanie pierwszej kopii
m = starmap(mult, krotki2)                    # iloczyny używające drugiej kopii
g = groupby(m, lambda x: len(str(x)))         # dzielenie ciągu liczb wg długości ich zapisu
#print(list(g))                                # to by popsuło wypisywanie poniżej
print(list(map(lambda p: (p[0], list(p[1])), g))) # wypisanie z odpakowaniem wewnętrznych iteratorków

Funkcja `groupby` tnie ciąg wejściowy na podciągi, cięcie następuje po każdej zmianie wartości funkcji podanej jako drugi argument.
Wynikiem jest iterator złożony z par: wartość funkcji, iterator odp. fragmentu ciągu wejściowego.

Używając funkcji `groupby` należy zwrócić uwagę na to, że wyjściowy iterator kawałków oraz same kawałki wspóldzielą wejściowy iterator! Dlatego należy korzystać z kawałków po kolei. Na przykład to nie zadziała:

In [ ]:
g = groupby([1,1,1,2,2,2,2,3,3,3])
(jeden, jedynki) = next(g)
# print(list(jedynki))  # tu działa
(dwa, dwojki) = next(g)
print(list(jedynki))   # a tu nie :)
print(list(dwojki))  # teraz działają tylko dwójki

Uwaga! Czasami rozdwajanie za pomocą `tee` też nie pomaga...

In [ ]:
g = groupby([10,11,12,21,22,23,24,31,32,34], lambda x: x // 10)
g1, g2 = tee(g) # rozdwojenie (?!)
#print(list(g1)) # Ale to też psuje !!!!
print(list(map(lambda p: (p[0], tuple(p[1])), g2)))

Dzieje się tak dlatego, że początkowy iterator, ten zrobiony z listy, jest współdzielony pomiędzy `g1` i `g2`. I to jego tak naprawdę wyczerpuje transformacja w listę.

### Ćwiczenie 1
Rozwiąż za pomocą `groupby`, `map` oraz `max` zadanie z o najdłuższym plateau w liście (czyli wyznacz długość najdłuższego spójnego fragmentu listy o równych wartościach), np. `plateau([1,1,1,2,2,2,2,1,1,3,4,5,5,5]) = 4`
Nie zapomnij o liście pustej!

In [ ]:
def plateau(arr):
    return  max(list(map(lambda x: len(tuple(x[1])), groupby(arr))), default = 'empty')

## Moduł 'operator'
W niniejszym scenariuszu, i w poprzednim też, napisaliśmy mnóstwo małych wyrażeń lambda. Część z nich polegała na zamianie w funkcję operacji takiej jak mnożenie czy dodawanie. Często tak naprawdę nie ma potrzeby robienia tego za pomocą wyrażeń lambda, ponieważ odpowiednie funkcje nazwane są zdefiniowane w module `operator`. Najprościej zacząć zapoznawanie się z tym modułem od [niniejszej tabelki](https://docs.python.org/3/library/operator.html#mapping-operators-to-functions). Przykład użycia:

In [ ]:
import operator
def silnia(n):
    return reduce(operator.mul, range(1,n+1), 1)
silnia(10)

Operatory zdefiniowane w module `operator` w większości nie wymagają dalszego wyjaśniania. Ale kilka omówimy :) Przydają się one nie tylko jako funkcje używane w `map`, `starmap` czy `reduce`, ale także jako element wyrażeń lambda. Dotyczy to zwłaszcza takich operatorów jak np. `setitem` czy `delitem`, które pozwalają na zastąpienie instrukcji (niedozwolonych w wyrażeniach lambda) wywołaniami funkcji. Na przykład w ten sposób można efektywnie (bez wielokrotnego kopiowania listy) "podwoić" elementy na liście za pomocą `reduce`:

In [ ]:
print(reduce(lambda l,x: operator.setitem(l, slice(len(l),None), [x,x]) or l, range(7), []))
#print(len(reduce(lambda l,x: operator.setitem(l, slice(len(l),None), [x,x]) or l, range(30_000), [])))
#print(len(reduce(lambda l,x: l+[x,x], range(30_000), [])))

Wywołanie `setitem` powyżej odpowiada instrukcji `l[len(l):] = [x,x]`, która powoduje dopisanie do końca listy dwóch elementów `x`. Oczywiście ten sam efekt można by uzyskać używając `l.extend([x,x])`, ale w przypadku potrzeby użycia bardziej subtelnych modyfikacji listy, wersja z `operator.setitem` jest jedyną możliwą.
Fragment `<wyrażenie dające None> or l` tak naprawdę ma na celu wywołanie funkcji, a "potem" oddanie wartości (zmodyfikowanego) `l`. Omawialiśmy tego rodzaju wyrażenia udające ciąg instrukcji w poprzednim scenariuszu.

Aby przekonać się o efektywności wersji przedłużającej listę w miejscu, dla odróżnienia od `l + [x,x]`, które konstruuje za każdym wywołaniem nową listę, wystarczy porównać czas wykonania dwóch zakomentowanych linii w powyższej kratce. Działają one tak samo, ale dla dłuższych list.

*Uwaga o stylu:* Na początku poprzedniego scenariusza o programowaniu funkcyjnym, wspomnieliśmy, że polega ono na tym, by nie korzystać z efektów ubocznych, ale by obliczać nowe wartości na podstawie starych. Tutaj łamiemy tę zasadę ze względu na efektywność: zmieniamy stan obiektu reprezentującego listę. W sensie ścisłym nie jest to już programowanie funkcyjne (czy deklaratywne). Jednak w Pythonie pozwalamy sobie na takie odejścia od ortodoksji, gdyż Python nie jest językiem funkcyjnym. Nie jest to też uważane za zły styl programowania, bo zmiany stanu mają charakter lokalny: zmieniamy stan listy, którą utworzyliśmy specjalnie do wykonania operacji reduce.
Inną sprawą byłoby zmienianie stanu jakiegoś globalnego obiektu - tego wciąż unikamy!

Inną funkcją, która mogłaby się przydać przy podwajaniu, jest `iadd`. Zmienia ona listę w miejscu i zwraca (zmienioną) listę. Operację `iadd` można rozumieć jako istotną część dodawania w miejsu `l += <wyr>`, które w całości jest mniej więcej tym co `l = iadd(l, <wyr>)`. Podwajanie z użyciem `iadd` można zapisać tak:

In [ ]:
reduce(lambda l,x: operator.iadd(l, [x,x]), range(0,7), [])

Samego `iadd` można użyć do efektywnego "rozpłaszczenia" listy list: 

In [ ]:
print(reduce(operator.iadd, [[1,2,3],[4,5,6],[],[7,8]], []))
# print(len(reduce(operator.iadd, ([n,n+1] for n in range(0, 40_000, 2)), [])))
# print(len(reduce(operator.add, ([n,n+1] for n in range(0, 40_000, 2)), [])))

O efektywności modyfikacji w miejscu w porównaniu ze zwykłym dodawaniem list (które tym razem można zastąpić przez `operator.add`) można się przekonać odkomentowując dwie linijki w powyższej kratce. Zamiast `add` i `iadd` można też użyć równoważnych `concat` i `iconcat`, których nazwy, w przeciwieństwie do `add` i `iadd`, sugerują łączenie sekwencji, a nie "jakieś" dodawanie :)

Kolejnym interesującym operatorem jest `itemgetter`. Odpowiada on w zasadzie `getitem`, przy czym argumenty ma w innej kolejności i w dodatku podzielone "na pół". Dokładniej, aby wykonać `a[i]` jako wywołanie funkcji, możemy napisać `getitem(a,i)`, albo `itemgetter(i)(a)` - wartość wyrażenia `itemgetter(i)` to funkcja, która jak dostanie parametr `a`, to zwróci `a[i]`. Przydaje się to np. w następujących okolicznościach:

In [ ]:
sorted([("ma", 3), ("kota",8), ("ala", 1), ("wściekłego", 6)], key=operator.itemgetter(1)) 

Argumentami funkcji `itemgetter` mogą być też slice'y i argumentów może być kilka - wynikiem jest wtedy tupla, np:

In [ ]:
operator.itemgetter(1, slice(4,6), -2)("ala ma kota")

Oprócz używania funkcji `itemgetter` do sekwencji, można jej używać do słowników - wtedy argumentami są obiekty będącymi nazwami pól, np:

In [ ]:
operator.itemgetter(1, "ala")({1: "kot", 18 : "a kuku", "ala" : 10, "kota" : 7})

Podobną funkcją jest `attrgetter`, ale żeby ją zademonstrować musimy wprowadzić typ nazwanych tupli:

In [ ]:
from collections import namedtuple
Punkt = namedtuple("Punkt", ["x", "y"])
Prostokąt = namedtuple("Prostokąt", ["ld", "pg"])

p = Punkt(10,13)
q = Punkt(y=18, x=17)
prost = Prostokąt(ld=p, pg=q)

print( operator.attrgetter("pg")(prost) )
print( operator.attrgetter("ld.x", "pg.y")(prost) )

Funkcja `attrgetter` dla argumentu postaci `"a.b.c"` zwraca funkcję, która dla obiektu `x` wyliczy `x.a.b.c`.

## Częściowa aplikacja
Uzyteczność operatorów może być znacznie zwiększona poprzez użycie funkcji `partial` z modułu `functools`.
Pozwala ona aplikować tylko część argumentów funkcji, a na pozostałe czekać, np:

In [ ]:
print(list( map(partial(operator.add, 1), [3,4,5]) ))
print(list( map(partial(operator.getitem, "ala ma kota"), [1, slice(4,6), -2]) ))
print(list( filter(partial(operator.lt, 0), range(-5,5)) ))

Ostatni przykład może być trochę mylący, bo nie chodzi o elementy, które są "lt od 0" tylko o te elementy `x`, które spełniają `lt 0 x`, czyli `0 < x` :)

Oczywiście trudno przekonywać, że definicje te, używające dość długich nazw funkcji są bardziej przejrzyste od krótkich wyrażeń lambda, ale warto wiedzieć o możliwości użycia funkcji `partial`, choćby w celu łatwiejszego przeczytania cudzego kodu.

### Ćwiczenie 2
Za pomoca `tee` `starmap`, `pow`, `chain` `partial` i `repeat` :) napisz funkcję, która iterator par `(a,b)` zamienia na iterator trójek `(a,b,c)` gdzie `c` to `a**b`. Staraj się (dla sportu :) nie używać wyrażeń lambda operujących na elementach iteratorów, zamiast tego używaj odopwiednich operatorów i ewentualnie funkcji `partial`.

Aby umieścić (w drugiej wersji rozwiązania) wszystko w jednym wyrażeniu lambda (choć będzie to zdecydowanie zbyt rozbudowane wyrażenie ;) możesz użyć następującego triku pozwalającego na kilkukrotne użycie wartości skomplikowanego podwyrażenia:  
`(lambda w: ..w..w.. )(podwyrażenie)`  
np.  
`(lambda c: c[0] + c + c[1])( ("ala m" + "a kota")[4:6] )`  
daje w wyniku napis `"mmaa"`.

In [ ]:
def newIterator(pair):
    return (pair[0], pair[1], operator.mul(pair[0],pair[1]))


## Generatory i iteratory dokładniej

Przez dużą część dzisiejszego scenariusza mówiliśmy o iteratorach, a na poprzednich zajęciach używaliśmy z kolei pojęcia generatora. Iterator to najbardziej abstrakcyjne pojęcie, oznacza dowolny obiekt ze stanem, z którego za pomocą wbudowanej procedury `next` można "wyciągać" kolejne elementy, aż do opróżnienia iteratora, które sygnalizowane jest odpowiednim wyjątkiem `StopIteration`. Zwykle nie używamy `next` bezpośrednio i nie łapiemy wyjątku, tylko robi to za nas np. `for`, albo `list(...)`. Generator z kolei to rodzaj iteratora, który jest specjalnym wyrażeniem lub funkcją.

Ostatnim pojęciem z tej rodzinki jest "obiekt iterowalny" (ang. *iterable*) i jest to dowolny obiekt, często struktura danych (lista, krotka, range, ...), może być bezstanowa, z której za pomocą `iter` można wyprodukować iterator.

Oczywiście każdy iterator też jest iterable, `iter` jest w tym przypadku po prostu identycznością.

Instrukcja `for x in <iterable>: <przetwórz x>` jest zaimplementowana mniej więcej tak:

In [ ]:
g = range(3) # <iterable>
i = iter(g)  # iterator ustawiony na początek
while True:
    try:
        x = next(i)  # próba pobrania kolejnego elementu z iteratora
    except StopIteration:
        break        # iterator opróżniony - wychodzimy z pętli
    print(x) # <przetwórz x>
# koniec

Podobnie wyglądałaby implementacja np. `list(<iterable>)`.

Bardzo ciekawa jest obserwacja "połączenia" iteratora ze zmienialną strukturą danych: 

In [ ]:
def może_next(i):
    try:
        print(next(i))
    except Exception as e:
        print("Wyjątek", type(e).__name__, e)

g = [0,1]
może_next(g)    # nie działa - to nie iterator :)
i = iter(g)     # to iterator
może_next(i)    # działa
może_next(i)    # teraz jesteśmy "na końcu iteratora", ale...
print("Powiększamy")
g += [2,3]
może_next(i)    # znów możemy czerpać :)
może_next(i)  
może_next(i)    # a teraz "przechodzimy przez koniec"...
print("Powiększamy")
g += [4,5]
może_next(i)    # i to oznacza definitywny koniec iterowania :(

### Generatory

Własny iterator możemy zbudować za pomocą *generator expression*:

In [ ]:
g = (x*x for x in range(4))
print(iter(g) is g)
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(tuple(g))  # opróżniony :(

Możemy go też zbudować za pomocą *generator function*, która "zamiast" ostatecznego `return` używa nieostateczne `yield`:

In [ ]:
def pierwsze(n):
    lista = []
    k = 2   # pierwszy kandydat
    while len(lista) < n:
        for p in lista:            # testujemy kandydata k
            if k % p == 0: break   # k odpadł
        else:                      # jeśli nie odpadł
            lista.append(k)        #   dodajemy go do listy
            yield(k)               #   pokazujemy światu
        k += 1                     # bierzemy kolejnego kandydata 

pierw = pierwsze(4)
print(next(pierw))
print(next(pierw))
print(next(pierw))
print(next(pierw))
print(tuple(pierw))

print(list(pierwsze(20)))

Taki generator to "jakby" osobny programik, z własnym lokalnym stanem, który wstrzymuje swoje działanie po każdym `yield` i czeka na ponowne "popchnięcie" poprzez `next`. W zasadzie `yield` (podobnie do `return`) nie musi mieć argumentu, "wyciągnięta" wartość to wtedy `None`. Historycznie, taki zawieszalny fragment programu nazywa się po polsku [*współprogramem*](https://pl.wikipedia.org/wiki/Wsp%C3%B3%C5%82program) (po angielsku *coroutine*).

**Uwaga!** Aby zawołać z wnętrza generatora inny generator (w celu oddania "jego" zawartości), należy użyć składni `yield from drugi_generator()`. 

In [ ]:
def liczby_pierwsze(*args):
    if len(args) == 1:
        #pierwsze(args[0])    # z tym nie zadziała!
        yield from pierwsze(args[0])  # a z tym tak!

print(list(liczby_pierwsze(10)))

Ciekawostka: do wstrzymanego (przez `yield`) generatora możemy przekazać jakiś obiekt, gdy zamiast `next` użyje się `send`. Generator odbiera ten obiekt poprzez złapanie wartości `yield`:

In [ ]:
def sumowanie():
    s = 0
    while True:
        x = yield s   # wysyłamy s, czekamy..... odbieramy x
        if x is None: # albo i nie odbieramy :)
            s += 1
        else:
            s += x

gen = sumowanie()
print(next(gen))  # równoważne z gen.send(None)
print(next(gen))
print(gen.send(10))
print(next(gen))
print(gen.send(100))
print(next(gen))

### Ćwiczenie 3
Drzewa binarne można w Pythonie reprezentować w następujący sposób: puste drzewo to `None`, niepuste drzewo, czyli takie, które składa się z jakiegoś elementu `el` umieszczonego w korzeniu oraz dwóch poddrzew to trójka `(lewe,el,prawe)`.

Napisz dwuargumentową funkcję `zrob_drzewo`, która z danego iterabla "wyciągnie" daną liczbę elementów (niekoniecznie wszystkie na raz!) i stworzy z nich drzewo zrównoważone.
Kolejność elementów w drzewie może być dowolna, byle sensowna (patrz niżej), ale najlepiej, żeby była to kolejność *inorder*. Czyli `zrob_drzewo(7, "alakota")` powinno dać drzewo  
`(((None,'a',None),'l',(None,'a',None)),'k',((None,'o',None),'t',(None,'a',None)))`

Napisz funkcję `obejdź` z parametrem "typu" drzewo oraz paramerami boolowskimi `preorder`, `inorder`, `postorder`, o domyślnych wartościach `False`, która:
* upewni się, że dokładnie jeden z trzech argumentów jest prawdą, reszta fałszem;
* wyprodukuje iterator po elementach drzewa w obejściu zgodnym z żądaną kolejnością.

Zrób to dwa razy: raz używając `yield` i `yield from`, a raz używając funkcji `chain` (i może `repeat`) z modułu `itertools`.

## Dekoratory

Na pewno w którymś momencie swojej Pythonowej kariery spotkacie się z jakąś taką `@dekoracją` przed definicją funkcji: 

In [ ]:
@cache
def fibC(n):
    print(f"debug: fibC({n})")
    return fibC(n-1) + fibC(n-2) if n >= 2 else n

print(f"Fib(4) to {fibC(4)}.")
print(f"Fib(6) to {fibC(6)}.")
print(f"Fib(5) to {fibC(5)}.")

Widać co tu się stało: dekoracja `@cache` spowodowała spamiętywanie wartości kolejnych wywołań funkcji `fibC`. Ale jak to się stało? Otóż składnia z "dekoratorem" jest lukrem syntaktycznym do czegoś takiego: 

In [ ]:
def fibL(n):
    print(f"debug: fibL({n})")
    return fibL(n-1) + fibL(n-2) if n >= 2 else n

fibL = cache(fibL)

A funkcja (wyższego rzędu) `cache` pochodzi z modułu `functools` i jest [opisana tutaj](https://docs.python.org/3/library/functools.html#functools.cache). Jej argumentem jest (oryginalna) funkcja, a wynikiem funkcja, która obsługuje cache, niekiedy wołając funkcję oryginalną. Uproszczona wersja tego dekoratora mogłaby wyglądać tak:

In [ ]:
def kesz(f):
    mem = {}
    def lepsza_f(n):
        if n not in mem: mem[n] = f(n)
        return mem[n]
    return lepsza_f

@kesz
def fibK(n):
    print(f"debug: fibK({n})")
    return fibK(n-1) + fibK(n-2) if n >= 2 else n

# To działa!
print(f"Fib(4) to {fibK(4)}.")
print(f"Fib(6) to {fibK(6)}.")
print(f"Fib(5) to {fibK(5)}.")

### Ćwiczenie 4
Napisz dekorator `@odpluskw`, który przed każdym wywołaniem dekorowanej funkcji wypisuje wszystkie argumenty wywołania, a po powrocie wynik.

In [ ]:
def odpluskw(f):
    def text(*args):
        print("Arguments >> ", args)
        result = f(*args)
        print("Result >> ", result)
        return result
    return text


# Zadanie dnia
Zadanie dnia polega na zaprogramowaniu rozwiązań trzech ostatnich ćwiczeń z niniejszego scenariusza:

2. iterator (a,b) -> iterator (a,b,a**b) za pomocą funkcji z modułów `itertools` i `operator`;
3. dekorator służacy do wypisywania parametrów wywołania funkcji i jej wyniku;
4. konwersja iterator -> drzewo i obejście drzewa by stworzyć iterator.